In [1]:
import numpy as np
import pandas as pd

def stockTree(underlyingPrice, upReturn, numPeriods):
    upMove = 1 + upReturn
    prices = [[underlyingPrice*upMove**(t-2*i) for i in range(t+1)] for t in range(numPeriods+1)]
    df = pd.DataFrame(prices).transpose()
    df.columns = [f"date {i}" for i in df.columns]
    return df 

def americanTree(underlyingPrice, strikePrice, interestRate, upReturn, numPeriods, optionType):
    assert optionType in ("call", "put"), "optionType must be call or put"
    def f(x):
        if optionType == "call":
            return np.maximum(np.array(x) - strikePrice, 0)
        else:
            return np.maximum(strikePrice - np.array(x), 0)
    upMove = 1 + upReturn
    downMove = 1 / upMove
    prob = (1+interestRate-downMove) / (upMove-downMove)
    pvFactor = 1 / (1 + interestRate)
    
    ## underlying price at last date
    ST = [underlyingPrice*upMove**(numPeriods-2*i) for i in range(numPeriods+1)]

    ## option price at last date
    x = f(ST)

    ## back up in the tree a period at a time
    values = [x]
    while len(x) > 1:

        ## discounted expected values
        x0 = pvFactor * (prob*x[:-1] + (1-prob)*x[1:])

        ## early exercise values
        t = len(x0) - 1
        St = [underlyingPrice * upMove**(t - 2 * i) for i in range(t + 1)]

        ## maximum of discounted expected values and early exercise values
        x = np.maximum(x0, f(St))

        ## store option values at first of lst
        values.insert(0, x)
    
    df = pd.DataFrame(values).transpose()
    df.columns = [f"date {i}" for i in df.columns]
    return df

#### Example input parameters

In [2]:
underlyingPrice = 50
strikePrice = 50
timeToMaturity = 1
numPeriods = 6
sigma = 0.4   # annual volatility
r = 0.04      # annual risk-free rate

#### Calculated parameters

In [3]:
dt = timeToMaturity / numPeriods
upReturn = np.exp(sigma*np.sqrt(dt)) - 1   # single-period up return
interestRate = np.log(1+r)*dt             # single-period interest rate

#### Stock tree

In [4]:
stock = stockTree(
    underlyingPrice=underlyingPrice,
    upReturn=upReturn,
    numPeriods=numPeriods
)

stock

,date 0,date 1,date 2,date 3,date 4,date 5,date 6
0,50.0,58.869452,69.312249,81.607482,96.083756,113.127962,133.195624
1,NaN,42.466847,50.000000,58.869452,69.312249,81.607482,96.083756
2,NaN,NaN,36.068661,42.466847,50.000000,58.869452,69.312249
3,NaN,NaN,NaN,30.634446,36.068661,42.466847,50.000000
4,NaN,NaN,NaN,NaN,26.018966,30.634446,36.068661
5,NaN,NaN,NaN,NaN,NaN,22.098869,26.018966
6,NaN,NaN,NaN,NaN,NaN,NaN,18.769385


#### American put tree

In [5]:
put = americanTree(
    underlyingPrice=underlyingPrice,
    strikePrice=strikePrice,
    interestRate=interestRate,
    upReturn=upReturn,
    numPeriods=numPeriods,
    optionType=optionType
)

put

NameError: name 'optionType' is not defined